In [42]:
import requests
import json
import hashlib

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

def generate_id(doc):
    doc_str = json.dumps(doc, sort_keys=True)  # Convert document to a JSON string
    doc_hash = hashlib.md5(doc_str.encode()).hexdigest()  # Generate an MD5 hash
    return doc_hash

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        doc_id = generate_id(doc)  # Generate a unique ID for the document
        doc['doc_id'] = doc_id  # Add the unique ID as a doc_id key
        documents.append(doc)

In [43]:
documents[0:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'doc_id': 'bae7a31e6abaddb52b4061dcf238fc61'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'doc_id': '3e5d4959603c68a1e154fa2a6bd9d1e8'}]

In [46]:
from elasticsearch import Elasticsearch

elasticsearch_host = "http://localhost:9200"
es = Elasticsearch(hosts=elasticsearch_host)
es.info()

ObjectApiResponse({'name': 'd83fffb93737', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'hzrR9d2WS_ShFByVhIC38Q', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [48]:
from elasticsearch.exceptions import NotFoundError, BadRequestError

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "dynamic": "strict",
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "doc_id": {"type": "keyword"}
        }
    }
}

index_name = "zoomcamp-courses-questions"

# Delete the existing index if it exists
try:
    es.indices.delete(index=index_name)
    print(f"Deleted index '{index_name}'")
except NotFoundError as e:
    print(f"Index '{index_name}' does not exist, no need to delete")
except BadRequestError as e:
    print(f"Error deleting index '{index_name}': {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Create the index with the new settings
try:
    response = es.indices.create(index=index_name, settings=index_settings["settings"], mappings=index_settings["mappings"])
    print(f"Created index '{index_name}': {response}")
except Exception as e:
    print(f"Error creating index '{index_name}': {e}")

response

Deleted index 'zoomcamp-courses-questions'
Created index 'zoomcamp-courses-questions': {'acknowledged': True, 'shards_acknowledged': True, 'index': 'zoomcamp-courses-questions'}


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'zoomcamp-courses-questions'})

In [49]:
from tqdm.auto import tqdm

for doc in tqdm(documents, ncols = 100):
    doc_id = doc["doc_id"]
    es.index(index=index_name, id=doc_id, document=doc)

es.count(index=index_name)

  0%|                                                                       | 0/948 [00:00<?, ?it/s]

ObjectApiResponse({'count': 743, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [50]:
question = "How do copy a file to a Docker container?"

In [51]:
def retrieve_documents(
        query,
        max_results=5,
        boost=4,
        es=Elasticsearch("http://localhost:9200"),
        index_name="zoomcamp-courses-questions",
        ):
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": [f"question^{boost}", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]

    return documents

In [52]:
retrieve_documents(query=question, max_results=3)

/var/folders/l2/z6b07fkj7y93gz1y8xnd1k600000gn/T/ipykernel_218/1421582577.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_query)


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp',
  'doc_id': '61d50f53fe4c63a2cd0ec015a612e7af'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy file

In [53]:
context_docs = retrieve_documents(query=question, max_results=3)

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc in context_docs:
    doc_str = context_template.format(question=doc['question'], text=doc['text'])
    context += doc_str + "\n"  # Add a newline between documents

context = context.strip()
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)
Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani
Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do

/var/folders/l2/z6b07fkj7y93gz1y8xnd1k600000gn/T/ipykernel_218/1421582577.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index=index_name, body=search_query)


In [54]:
prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

len(prompt)

1444

In [55]:
!pip install tiktoken


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [61]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

tokens = encoding.encode(prompt)

print(f"Number of tokens: {len(tokens)}\n")

print(f"Tokens decoded:")
for token in tokens:
    print(f"{token}: {encoding.decode([token])}")

Number of tokens: 320

Tokens decoded:
63842: You're
261:  a
4165:  course
14029:  teaching
29186:  assistant
13: .
30985:  Answer
290:  the
150339:  QUESTION
4122:  based
402:  on
290:  the
31810:  CONT
8099: EXT
591:  from
290:  the
40251:  FAQ
7862:  database
558: .

8470: Use
1606:  only
290:  the
19719:  facts
591:  from
290:  the
31810:  CONT
8099: EXT
1261:  when
55959:  answering
290:  the
150339:  QUESTION
364: .


107036: QUESTION
25: :
3253:  How
621:  do
5150:  copy
261:  a
1974:  file
316:  to
261:  a
91238:  Docker
9282:  container
1715: ?


10637: CON
50738: TEXT
734: :

48: Q
25: :
3253:  How
621:  do
357:  I
15199:  debug
261:  a
62275:  docker
9282:  container
3901: ?

32: A
25: :
41281:  Launch
290:  the
9282:  container
3621:  image
306:  in
25383:  interactive
6766:  mode
326:  and
151187:  overriding
290:  the
7251:  entry
4859: point
11: ,
813:  so
484:  that
480:  it
13217:  starts
261:  a
38615:  bash
6348:  command
558: .

68923: docker
2461:  run
533:  -
278:

In [62]:
import os
from dotenv import load_dotenv
from openai import OpenAI

env_file_path = ".env"
load_dotenv(dotenv_path=env_file_path, verbose=True, override=True)

open_ai_api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=open_ai_api_key)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


In [63]:
input_price_per_token_gpt_4o = 2.5/1000000
output_price_per_token_gpt_4o = 10/1000000
num_requests = 1000
avg_per_request_tokens = 150
avg_output_tokens = 250

total_cost = (num_requests * avg_per_request_tokens * input_price_per_token_gpt_4o + num_requests * avg_output_tokens * output_price_per_token_gpt_4o)

print(f"Total cost: ${total_cost:.2f}")

Total cost: $2.88
